In [0]:
from google.colab import drive
drive.mount('/content/drive')
SEED = 42

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#### MAIN

In [0]:
## main
def main():
    _process_data()
    _prepare_data()
#     _load_merge_data()
    _build_model()
    _train_predict_amount()
    _train_predict_survival()
    _generate_submission()
    

if __name__ == "__main__":
    
    testname = 'test1'
    print(('*'* 50) +'\ntry model to test1 data\n' + ('*'*50))
    main()
    print('\n\n\nThe end.\n\n\n')
    
#     testname = 'test2'
#     print(('*'* 50) +'\ntry model to test2 data\n' + ('*'*50))
#     main()
#     print('\n\n\nThe end.\n\n\n')

**************************************************
try model to test2 data
**************************************************
CODE VERSION :  try 7 - delete and merge variables


STEP1: processing data ...

Loading data ...

Grouping activity data ...

 + making day variables for train activity

 + making day variables for test activity
activity id unique and shape :
40000 (40000, 20)
20000 (20000, 20)

Grouping payment data ...
payment id unique and shape :
23726 (23726, 3)
9499 (9499, 3)

Grouping combat data ...
combat id unique and shape :
40000 (40000, 21)
20000 (20000, 21)

Grouping pledge data ...
pledge id unique and shape :
33854 (33854, 15)
16263 (16263, 15)

Grouping trade data ...
trade id unique and shape :
60645 (60645, 16) 72300 (72300, 16)
36675 (36675, 16) 41107 (41107, 16)

Merging all data ...

merge id unique and shape :
40000 (40000, 59)
20000 (20000, 57)
test and train merge data are saved on drive/My Drive/bigcontest2019/scripts/model/metrics/nes_inference//merge

#### try 7 - delete and merge variables 
(2019-08-30 09:43:50 PM	6772.33	2496.24	9268.57)

In [0]:

# -*- coding: utf-8 -*-


from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    global codeversion
    
    codeversion = 'try 7 - delete and merge variables'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value
        
    
## STEP1: process data    
def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
    print('NA ratio: ')
    print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)
    
def encode_features(df):
    print('\nEncoding features ...')
    
    for feature in df:
        if df[feature].dtype == 'object':
            print('Encoding ', feature)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[feature].values))
            df[feature] = lbl.transform(list(df[feature].values))
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()
    
def _process_data():
    
    print('CODE VERSION : ', codeversion)
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

        
    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')

    
    
    print('\nGrouping activity data ...')

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test_activity['game_money_change_minus'] = test_activity['game_money_change']
    test_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test_activity['game_money_change_minus']]
    test_activity['game_money_change'] = [0 if i < 0 else i for i in test_activity['game_money_change']]

    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    acc_id_list = train_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []
    
    print('\n + making day variables for train activity')

    for i in acc_id_list :
        dayunique = train_activity[train_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    train_activity_merge = train_activity_merge.merge(all_day, on = ['acc_id'], how ='left')
    
    print('\n + making day variables for test activity')

    acc_id_list = test_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []

    for i in acc_id_list :
        dayunique = test_activity[test_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    test_activity_merge = test_activity_merge.merge(all_day, on = ['acc_id'], how ='left')
    
    
    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
#     train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
#     test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

#     train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량으로 변경
#     test_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)



    train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'mean'}).reset_index(drop=True)
    test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'mean'}).reset_index(drop=True)

    train_payment_merge.rename(columns={'amount_spent':'mean_spent'}, inplace=True) #amout_spent를 일일 평균 구매량으로 변경
    test_payment_merge.rename(columns={'amount_spent':'mean_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
  
    train_combat_merge.rename(columns=combat_rename, inplace=True)
    test_combat_merge.rename(columns=combat_rename, inplace=True)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    test_pledge_merge = test_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    
    
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    train_source_trade, on = ['acc_id'], how = 'left').merge(
    train_target_trade, on = ['acc_id'], how = 'left')
        
    test_merge = test_activity_merge.merge(
    test_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    test_source_trade, on = ['acc_id'], how = 'left').merge(
    test_target_trade, on = ['acc_id'], how = 'left')
    
    train_merge['attack'] = (train_merge['random_attacker_cnt']+train_merge['combat_random_attacker_cnt']) -(train_merge['random_defender_cnt'] + train_merge['combat_random_defender_cnt'])
    test_merge['attack'] = (test_merge['random_attacker_cnt']+test_merge['combat_random_attacker_cnt']) -(test_merge['random_defender_cnt'] + test_merge['combat_random_defender_cnt'])
    
    train_merge['cnt'] = train_merge['temp_cnt']+train_merge['same_pledge_cnt']+train_merge['etc_cnt'] + train_merge['combat_temp_cnt']+train_merge['combat_same_pledge_cnt']+train_merge['combat_etc_cnt']
    test_merge['cnt'] = test_merge['temp_cnt']+test_merge['same_pledge_cnt']+test_merge['etc_cnt'] + test_merge['combat_temp_cnt']+test_merge['combat_same_pledge_cnt']+test_merge['combat_etc_cnt']
    
    
    train_merge['pledge_ratio'] = train_merge['combat_char_cnt']/train_merge['play_char_cnt']
    test_merge['pledge_ratio'] = test_merge['combat_char_cnt']/test_merge['play_char_cnt']
  
    train_merge['day_spent'] = train_merge['day']*train_merge['mean_spent']
    test_merge['day_spent'] = test_merge['day']*test_merge['mean_spent']
    
    drop_merge = ['day','fishing','revive','elf','magician','warrior','dark-elf','knight','dragon','illusionist','class','random_attacker_cnt',
                  'random_defender_cnt','temp_cnt','same_pledge_cnt','etc_cnt','same_pledge_cnt','num_opponent','combat_temp_cnt', 
                  'combat_same_pledge_cnt', 'combat_etc_cnt','combat_random_defender_cnt','combat_random_attacker_cnt','playtime',
                  'combat_char_cnt','play_char_cnt','mean_spent']
    
    
    
    train_merge = train_merge.drop(drop_merge, axis = 1)
    test_merge = test_merge.drop(drop_merge, axis = 1)
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)
    
    train_merge.to_csv(submission_path + '/merge/' + 'train_merge.csv',index=False)  
    test_merge.to_csv(submission_path + '/merge/' + testname + '_merge.csv',index=False)  
    print('test and train merge data are saved on ' + submission_path + '/merge/' )
    
    
def _load_merge_data(): 

    train_merge = pd.read_csv(submission_path + 'merge/' + 'train_merge.csv')
    test_merge = pd.read_csv(submission_path + 'merge/' + testname + '_merge.csv')
  
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)

    
    # fill NA
#     fill_NA(train_merge)
    
    # encode features
#     encode_features(train_merge)
    
### STEP1.1 : prepare train and vaild data



def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape)
    print('train ys shape: ', train_ys.shape)
    
    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape)
    print('valid ys shape: ', valid_ys.shape)
    
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test_merge
    test_acc_id = test_merge['acc_id']
    test_x.drop(columns=['acc_id'], inplace=True)
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params

    xgb_params = {
        'eta': 0.007,
        'max_depth': 8, 
        'subsample': 0.8,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }

    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=5000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=100)
    

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=5000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=20)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']] # 마이너스값 0 처리 
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']] # 1 미만 값 1 처리
    
    #testname에 따라 amountspent value 키우기
    if testname == 'test1' :
        test_predict['amount_spent'] = test_predict['amount_spent']*18 #best : 20
    elif testname == 'test2' :
        test_predict['amount_spent'] = test_predict['amount_spent']*5
    else :
        print('testname is wrong!')
        
    test_predict.to_csv(submission_path + testname + '_predict.csv',index=False)    
 

#### ver1.6 - Day 변수 추가 (login_day_max, logout_day)

In [0]:

# -*- coding: utf-8 -*-


from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    global codeversion
    
    codeversion = 'ver1.6 - Day 변수 추가 (login_day_max, logout_day)'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value
        
    
## STEP1: process data    
def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
    print('NA ratio: ')
    print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)
    
def encode_features(df):
    print('\nEncoding features ...')
    
    for feature in df:
        if df[feature].dtype == 'object':
            print('Encoding ', feature)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[feature].values))
            df[feature] = lbl.transform(list(df[feature].values))
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()
    
def _process_data():
    
    print('CODE VERSION : ', codeversion)
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

        
    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')

    
    # merge payment
    print('\nGrouping activity data ...')

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test_activity['game_money_change_minus'] = test_activity['game_money_change']
    test_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test_activity['game_money_change_minus']]
    test_activity['game_money_change'] = [0 if i < 0 else i for i in test_activity['game_money_change']]

    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    acc_id_list = train_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []
    
    print('\n + making day variables for train activity')

    for i in acc_id_list :
        dayunique = train_activity[train_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    train_activity_merge = train_activity_merge.merge(all_day, on = ['acc_id'], how ='left')
    
    print('\n + making day variables for test activity')

    acc_id_list = test_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []

    for i in acc_id_list :
        dayunique = test_activity[test_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    test_activity_merge = test_activity_merge.merge(all_day, on = ['acc_id'], how ='left')
    
    
    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
    train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
    test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

    train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량으로 변경
    test_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
  
    train_combat_merge.rename(columns=combat_rename, inplace=True)
    test_combat_merge.rename(columns=combat_rename, inplace=True)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    test_pledge_merge = test_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    
    
    # merge trade 
    print('\nGrouping trade data ...')

    # item_type dummies             
    train_item_type_dummy = pd.get_dummies(train_trade['item_type'])
    test_item_type_dummy = pd.get_dummies(test_trade['item_type'])

    # trade_type dummies
    type_names = {0:'personal_shop', 1:'trade_shop'}
    for elem in train_trade['type'].unique():train_trade[type_names[elem]] = train_trade['type'] == elem
    for elem in test_trade['type'].unique():test_trade[type_names[elem]] = test_trade['type'] == elem

    # merge dummies
    
    train_trade = train_trade.drop(columns=['type','item_type'])
    train_trade = pd.concat([train_trade, train_item_type_dummy], axis = 1)
    
    test_trade = test_trade.drop(columns=['type','item_type'])
    test_trade = pd.concat([test_trade, test_item_type_dummy], axis = 1)

    # rename columns
    trade_rename = {'day':'trade_day','server' : 'trade_server', 'time' : 'trade_time'}
    train_trade.rename(columns=trade_rename, inplace=True)
    test_trade.rename(columns=trade_rename, inplace=True)

    #divide source and target id
    
    train_source_trade = train_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    train_source_trade['source'] = 1
    train_target_trade = train_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    train_target_trade['target'] = 1
    
    test_source_trade = test_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    test_source_trade['source'] = 1
    test_target_trade = test_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    test_target_trade['target'] = 1


    #aggregate
    trade_source_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'source' : 'sum'}

    trade_target_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'target' : 'sum'}

    train_source_trade = train_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    train_target_trade = train_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)
    
    test_source_trade = test_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    test_target_trade = test_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)

    print('trade id unique and shape :')
    print(train_source_trade['acc_id'].nunique(), train_source_trade.shape, train_target_trade['acc_id'].nunique(), train_target_trade.shape)
    print(test_source_trade['acc_id'].nunique(), test_source_trade.shape, test_target_trade['acc_id'].nunique(), test_target_trade.shape)
    
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    train_source_trade, on = ['acc_id'], how = 'left').merge(
    train_target_trade, on = ['acc_id'], how = 'left')
        
    test_merge = test_activity_merge.merge(
    test_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    test_source_trade, on = ['acc_id'], how = 'left').merge(
    test_target_trade, on = ['acc_id'], how = 'left')
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)
    
    
    train_merge.to_csv(submission_path + '/merge/' + 'train_merge.csv',index=False)  
    test_merge.to_csv(submission_path + '/merge/' + testname + '_merge.csv',index=False)  
    print('test and train merge data are saved on ' + submission_path + '/merge/' )
    
    
def _load_merge_data(): 

    train_merge = pd.read_csv(submission_path + 'merge/' + 'train_merge.csv')
    test_merge = pd.read_csv(submission_path + 'merge/' + testname + '_merge.csv')
    
    train_merge['cnt'] = train_merge['random_attacker_cnt']+train_merge['random_defender_cnt']+train_merge['temp_cnt']+train_merge['same_pledge_cnt']+train_merge['etc_cnt']
    test_merge['cnt'] = test_merge['random_attacker_cnt']+test_merge['random_defender_cnt']+test_merge['temp_cnt']+test_merge['same_pledge_cnt']+test_merge['etc_cnt']
    
    train_merge = train_merge.drop(['day','fishing','revive','elf','magician','warrior','dark-elf','knight','dragon','illusionist','class',
                     'random_attacker_cnt','random_defender_cnt','temp_cnt','same_pledge_cnt','etc_cnt'], axis = 1)
    test_merge = test_merge.drop(['day','fishing','revive','elf','magician','warrior','dark-elf','knight','dragon','illusionist','class',
                     'random_attacker_cnt','random_defender_cnt','temp_cnt','same_pledge_cnt','etc_cnt'], axis = 1)
       
    print('test and train merge data are loaded on ' + submission_path + 'merge/' )
    
    # fill NA
#     fill_NA(train_merge)
    
    # encode features
#     encode_features(train_merge)
    
### STEP1.1 : prepare train and vaild data

def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape)
    print('train ys shape: ', train_ys.shape)
    
    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape)
    print('valid ys shape: ', valid_ys.shape)
    
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test_merge
    test_acc_id = test_merge['acc_id']
    test_x.drop(columns=['acc_id'], inplace=True)
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params

    xgb_params = {
        'eta': 0.007,
        'max_depth': 8, 
        'subsample': 0.8,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }

    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=10000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=20)
    

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=10000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=20)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']] # 마이너스값 0 처리 
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']] # 1 미만 값 1 처리
    
    #testname에 따라 amountspent value 키우기
    if testname == 'test1' :
        test_predict['amount_spent'] = test_predict['amount_spent']*18 #best : 20
    elif testname == 'test2' :
        test_predict['amount_spent'] = test_predict['amount_spent']*5
    else :
        print('testname is wrong!')
        
    test_predict.to_csv(submission_path + testname + '_predict.csv',index=False)    
 

### history code - DO NOT MODIFY

#### ver1.5 CODE - xgb with gridsearchCV (-50000점^^;)

In [0]:
#ref : https://www.kaggle.com/cuongvn08/xgboost-cross-validation-clean-coding-lb-0-64x

# -*- coding: utf-8 -*-

from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb

from datetime import datetime
import gc


from math import sqrt


import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    
    testname = 'test1'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value
        
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()
    
def _process_data():
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

        
    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')

    
    # merge payment
    print('\nGrouping activity data ...')

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test_activity['game_money_change_minus'] = test_activity['game_money_change']
    test_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test_activity['game_money_change_minus']]
    test_activity['game_money_change'] = [0 if i < 0 else i for i in test_activity['game_money_change']]

    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
    train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
    test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

    train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량인 max로 변경
    test_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
  
    train_combat_merge.rename(columns=combat_rename, inplace=True) #combat와 pledge value구분
    test_combat_merge.rename(columns=combat_rename, inplace=True)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True) 
    test_pledge_merge = test_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    
    
    # merge trade 
    print('\nGrouping trade data ...')

    # item_type dummies             
    train_item_type_dummy = pd.get_dummies(train_trade['item_type']) #item 더미변수 생성
    test_item_type_dummy = pd.get_dummies(test_trade['item_type'])

    # trade_type dummies
    type_names = {0:'personal_shop', 1:'trade_shop'}  #personal_shop, trade_shop구분하는 더미변수 생성
    for elem in train_trade['type'].unique():train_trade[type_names[elem]] = train_trade['type'] == elem
    for elem in test_trade['type'].unique():test_trade[type_names[elem]] = test_trade['type'] == elem

    # merge dummies
    
    train_trade = train_trade.drop(columns=['type','item_type'])
    train_trade = pd.concat([train_trade, train_item_type_dummy], axis = 1)
    
    test_trade = test_trade.drop(columns=['type','item_type'])
    test_trade = pd.concat([test_trade, test_item_type_dummy], axis = 1)

    # rename columns
    trade_rename = {'day':'trade_day','server' : 'trade_server', 'time' : 'trade_time'} #day, server, time 구분
    train_trade.rename(columns=trade_rename, inplace=True)
    test_trade.rename(columns=trade_rename, inplace=True)

    #divide source and target id, source 및 target 횟수 카운트 할 수 있도록 변수 생성 후 merge
    
    train_source_trade = train_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    train_source_trade['source'] = 1
    train_target_trade = train_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    train_target_trade['target'] = 1
    
    test_source_trade = test_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    test_source_trade['source'] = 1
    test_target_trade = test_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    test_target_trade['target'] = 1


    #aggregate
    trade_source_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'source' : 'sum'}

    trade_target_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'target' : 'sum'}

    train_source_trade = train_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    train_target_trade = train_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)
    
    test_source_trade = test_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    test_target_trade = test_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)

    print('trade id unique and shape :')
    print(train_source_trade['acc_id'].nunique(), train_source_trade.shape, train_target_trade['acc_id'].nunique(), train_target_trade.shape)
    print(test_source_trade['acc_id'].nunique(), test_source_trade.shape, test_target_trade['acc_id'].nunique(), test_target_trade.shape)
    
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    train_source_trade, on = ['acc_id'], how = 'left').merge(
    train_target_trade, on = ['acc_id'], how = 'left')
        
    test_merge = test_activity_merge.merge(
    test_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    test_source_trade, on = ['acc_id'], how = 'left').merge(
    test_target_trade, on = ['acc_id'], how = 'left')
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)
    
            
    def fill_NA(df):
        print('\nFilling NA ...')

        na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
#         print('NA ratio: ')
#         print(na_ratio) 

        for feature in df:
            if df[feature].dtype == 'object':
                df[feature] = df[feature].fillna("None") #fill.na option. object type인 경우 None으로, 그 외는 0으로 설정
            else:
                df[feature] = df[feature].fillna(0)

    def encode_features(df):
        print('\nEncoding features ...')

        for feature in df:
            if df[feature].dtype == 'object':
                print('Encoding ', feature)
                lbl = preprocessing.LabelEncoder()
                lbl.fit(list(df[feature].values))
                df[feature] = lbl.transform(list(df[feature].values))
    #fill NA
    fill_NA(train_merge)
    fill_NA(test_merge)
    
    #encode features
    encode_features(train_merge)
    encode_features(test_merge)
    
    
    
    
    
    
### STEP1.1 : prepare train and vaild data

def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global X

    global X_train
    global X_test
    
    global ya_train
    global ya_test
    
    global ys_train
    global ys_test
    
    global X_test_dataset
    global test_acc_id
    
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    
    ya = train_merge['amount_spent'].values
    ys = train_merge['survival_time'].values
    
    train_merge.drop(columns=drop_vars, inplace=True)
    X = train_merge.values
    
    test_acc_id = test_merge['acc_id']
    test_merge.drop(columns=['acc_id'], inplace=True)
    X_test_dataset = test_merge.values

    X_train, X_test, ya_train, ya_test = train_test_split(X, ya, test_size=1/5, random_state = SEED) #test size = 1/10
    X_train, X_test, ys_train, ys_test = train_test_split(X, ys, test_size=1/5, random_state = SEED) #test size = 1/10

    
    print('train x shape: ', X.shape)
    print('train ya shape: ', ya_train.shape)
    print('train ys shape: ', ys_train.shape)
    
    print('valid x shape: ', X_test.shape)
    print('valid ya shape: ', ya_test.shape)
    print('valid ys shape: ', ys_test.shape)
    
    print('test x shape: ', X_test_dataset.shape)   
    
   
    
    
    
    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    XGB_regressor = xgb.XGBRegressor()
    
    global XGB_hyper_params
    
    parameters = {'learning_rate': [0.07, 0.1, 0.3],
                  'max_depth': [3, 5, 7],
                  'n_estimators': [200, 400, 500]}

    XGB_hyper_params = GridSearchCV(estimator=XGB_regressor, param_grid=parameters, n_jobs=-1, cv=5)
    
   
    
## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global XGB_model_a
    global y_XGB_predict_a
    
    if testname == 'test2' : 
        XGB_model_a = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=SEED,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
                
    elif testname == 'test1' :
        XGB_hyper_params.fit(X_train, ya_train)
        print('\nXGB_hyper_params.best_params_ of amount :')
        print(XGB_hyper_params.best_estimator_) # find out the best hyper parameters
    
        XGB_model_a = XGB_hyper_params.best_estimator_
    else :
        print('testname is wrong!')

    XGB_model_a.fit(X_train, ya_train)
    
    %%time
    y_XGB_predict_a = XGB_model_a.predict(X_test)
   
    # let's plot feature_importance 
    print('\nDraw feature importance of amount_spent...')
    sns.set_style('ticks')
    fig, ax = plt.subplots()
    sns.barplot(y=list(train_merge.columns), x=list(XGB_model_a.feature_importances_)).set_title('feature_importances_ of amount_spent')
    fig.set_size_inches(20, 20)
    sns.despine()
    plt.show()
    
    
    
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global XGB_model_s
    global y_XGB_predict_s
    
    if testname == 'test2' :
        XGB_model_s = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=SEED,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
        
    elif testname == 'test1' :
        XGB_hyper_params.fit(X_train, ys_train)
        print('\nXGB_hyper_params.best_params_ of amount :')
        print(XGB_hyper_params.best_estimator_) # find out the best hyper parameters
        
        XGB_model_a = XGB_hyper_params.best_estimator_
    else :
        print('testname is wrong!')    
    
    XGB_model_s.fit(X_train, ys_train)
    %%time
    y_XGB_predict_s = XGB_model_s.predict(X_test)

    # let's plot feature_importance 
    print('\nDraw feature importance of survival_time...')
    sns.set_style('ticks')
    fig, ax = plt.subplots()
    sns.barplot(y=list(train_merge.columns), x=list(XGB_model_s.feature_importances_)).set_title('feature_importances_ of survival_time')
    fig.set_size_inches(20, 20)
    sns.despine()
    plt.show()


## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    ya_XGB_amount = np.expm1(XGB_model_a.predict(X_test_dataset))
    ys_XGB_amount = np.expm1(XGB_model_s.predict(X_test_dataset))

    # submitting our predictions
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = ys_XGB_amount
    test_predict['amount_spent'] = ya_XGB_amount
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']]
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']]

    
    # testname에 따라 amountspent value 키우기
    if testname == 'test1' :
        test_predict['amount_spent'] = test_predict['amount_spent']*20
        print('amount * 20')
#     elif testname == 'test2' :
#         test_predict['amount_spent'] = test_predict['amount_spent']*5
#         print('amount * 5')
    else :
        print('eveything is fine!')
    
    test_predict.to_csv(submission_path + testname + '_predict.csv',index=False) 
    print('\nsave completed.\n')
    print(testname + '_predict id unique and shape :')
    print(test_predict['acc_id'].nunique(), test_predict.shape)

#### ver1.4 CODE - merge trade (4102.32	2496.54	6598.863)

In [0]:
#ref : https://www.kaggle.com/cuongvn08/xgboost-cross-validation-clean-coding-lb-0-64x

# -*- coding: utf-8 -*-


from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    global codeversion
    
    codeversion = 'ver1.4 - simple merge trade'
    
#     testname = 'test1'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value
        
    
## STEP1: process data    
def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
    print('NA ratio: ')
    print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)
    
def encode_features(df):
    print('\nEncoding features ...')
    
    for feature in df:
        if df[feature].dtype == 'object':
            print('Encoding ', feature)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[feature].values))
            df[feature] = lbl.transform(list(df[feature].values))
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()
    
def _process_data():
    
    print('CODE VERSION : ', codeversion)
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

        
    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')

    
    # merge payment
    print('\nGrouping activity data ...')

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test_activity['game_money_change_minus'] = test_activity['game_money_change']
    test_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test_activity['game_money_change_minus']]
    test_activity['game_money_change'] = [0 if i < 0 else i for i in test_activity['game_money_change']]

    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
    train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
    test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

    train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량으로 변경
    test_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
  
    train_combat_merge.rename(columns=combat_rename, inplace=True)
    test_combat_merge.rename(columns=combat_rename, inplace=True)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    test_pledge_merge = test_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    
    
    # merge trade 
    print('\nGrouping trade data ...')

    # item_type dummies             
    train_item_type_dummy = pd.get_dummies(train_trade['item_type'])
    test_item_type_dummy = pd.get_dummies(test_trade['item_type'])

    # trade_type dummies
    type_names = {0:'personal_shop', 1:'trade_shop'}
    for elem in train_trade['type'].unique():train_trade[type_names[elem]] = train_trade['type'] == elem
    for elem in test_trade['type'].unique():test_trade[type_names[elem]] = test_trade['type'] == elem

    # merge dummies
    
    train_trade = train_trade.drop(columns=['type','item_type'])
    train_trade = pd.concat([train_trade, train_item_type_dummy], axis = 1)
    
    test_trade = test_trade.drop(columns=['type','item_type'])
    test_trade = pd.concat([test_trade, test_item_type_dummy], axis = 1)

    # rename columns
    trade_rename = {'day':'trade_day','server' : 'trade_server', 'time' : 'trade_time'}
    train_trade.rename(columns=trade_rename, inplace=True)
    test_trade.rename(columns=trade_rename, inplace=True)

    #divide source and target id
    
    train_source_trade = train_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    train_source_trade['source'] = 1
    train_target_trade = train_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    train_target_trade['target'] = 1
    
    test_source_trade = test_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    test_source_trade['source'] = 1
    test_target_trade = test_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    test_target_trade['target'] = 1


    #aggregate
    trade_source_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'source' : 'sum'}

    trade_target_agg = {'trade_day':'nunique', 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'target' : 'sum'}

    train_source_trade = train_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    train_target_trade = train_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)
    
    test_source_trade = test_source_trade.groupby('acc_id', as_index = False).agg(trade_source_agg).reset_index(drop=True)
    test_target_trade = test_target_trade.groupby('acc_id', as_index = False).agg(trade_target_agg).reset_index(drop=True)

    print('trade id unique and shape :')
    print(train_source_trade['acc_id'].nunique(), train_source_trade.shape, train_target_trade['acc_id'].nunique(), train_target_trade.shape)
    print(test_source_trade['acc_id'].nunique(), test_source_trade.shape, test_target_trade['acc_id'].nunique(), test_target_trade.shape)
    
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    train_source_trade, on = ['acc_id'], how = 'left').merge(
    train_target_trade, on = ['acc_id'], how = 'left')
        
    test_merge = test_activity_merge.merge(
    test_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    test_source_trade, on = ['acc_id'], how = 'left').merge(
    test_target_trade, on = ['acc_id'], how = 'left')
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)
    
    
    # fill NA
#     fill_NA(train_merge)
    
    # encode features
#     encode_features(train_merge)
    
### STEP1.1 : prepare train and vaild data

def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape)
    print('train ys shape: ', train_ys.shape)
    
    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape)
    print('valid ys shape: ', valid_ys.shape)
    
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test_merge
    test_acc_id = test_merge['acc_id']
    test_x.drop(columns=['acc_id'], inplace=True)
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params

    xgb_params = {
        'eta': 0.007,
        'max_depth': 8, 
        'subsample': 0.8,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }

    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=10000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=20)
    

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=10000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=20)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']] # 마이너스값 0 처리 
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']] # 1 미만 값 1 처리
    
    #testname에 따라 amountspent value 키우기
    if testname == 'test1' :
        test_predict['amount_spent'] = test_predict['amount_spent']*30
    elif testname == 'test2' :
        test_predict['amount_spent'] = test_predict['amount_spent']*5
    else :
        print('testname is wrong!')
        
    test_predict.to_csv(submission_path + testname + '_predict.csv',index=False)    
 


#### CODE - predict for testname

In [0]:
#ref : https://www.kaggle.com/cuongvn08/xgboost-cross-validation-clean-coding-lb-0-64x

# -*- coding: utf-8 -*-

from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    
    testname = 'test2'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value
        
    
## STEP1: process data    
def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
    print('NA ratio: ')
    print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)
    
def encode_features(df):
    print('\nEncoding features ...')
    
    for feature in df:
        if df[feature].dtype == 'object':
            print('Encoding ', feature)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[feature].values))
            df[feature] = lbl.transform(list(df[feature].values))
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()
    
def _process_data():
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

        
    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')

    
    # merge payment
    print('\nGrouping activity data ...')

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test_activity['game_money_change_minus'] = test_activity['game_money_change']
    test_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test_activity['game_money_change_minus']]
    test_activity['game_money_change'] = [0 if i < 0 else i for i in test_activity['game_money_change']]

    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_merge = test_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
    train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
    test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

    train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량으로 변경
    test_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
  
    train_combat_merge.rename(columns=combat_rename, inplace=True)
    test_combat_merge.rename(columns=combat_rename, inplace=True)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    test_pledge_merge = test_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left')
        
    test_merge = test_activity_merge.merge(
    test_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left')
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)
    
    
    # fill NA
#     fill_NA(train_merge)
    
    # encode features
#     encode_features(train_merge)
    
### STEP1.1 : prepare train and vaild data

def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape)
    print('train ys shape: ', train_ys.shape)
    
    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape)
    print('valid ys shape: ', valid_ys.shape)
    
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test_merge
    test_acc_id = test_merge['acc_id']
    test_x.drop(columns=['acc_id'], inplace=True)
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params
    xgb_params = {
        'eta': 0.007,
        'max_depth': 6, 
        'subsample': 0.6,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }
    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=10000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=10)

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=10000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=10)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']]
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']]
    
    test_predict.to_csv(submission_path + testname + '_predict.csv',index=False)    
 

#### CODE - predict for single test1

In [0]:
#ref : https://www.kaggle.com/cuongvn08/xgboost-cross-validation-clean-coding-lb-0-64x

# -*- coding: utf-8 -*-

from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path

    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value
        
    
## STEP1: process data    
def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
    print('NA ratio: ')
    print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)
    
def encode_features(df):
    print('\nEncoding features ...')
    
    for feature in df:
        if df[feature].dtype == 'object':
            print('Encoding ', feature)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[feature].values))
            df[feature] = lbl.transform(list(df[feature].values))
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()
    
def _process_data():
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test1_merge
        
    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test1_activity = pd.read_csv(test_path + 'test1_activity.csv')
    test1_combat = pd.read_csv(test_path + 'test1_combat.csv')
    test1_payment = pd.read_csv(test_path + 'test1_payment.csv')
    test1_pledge = pd.read_csv(test_path + 'test1_pledge.csv')
    test1_trade = pd.read_csv(test_path + 'test1_trade.csv')

    
    # merge payment
    print('\nGrouping activity data ...')

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test1_activity['game_money_change_minus'] = test1_activity['game_money_change']
    test1_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test1_activity['game_money_change_minus']]
    test1_activity['game_money_change'] = [0 if i < 0 else i for i in test1_activity['game_money_change']]

    activity_agg = {
        'day':'nunique', 'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', 
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_merge = train_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    test1_activity_merge = test1_activity.groupby('acc_id', as_index = False).agg(activity_agg).reset_index(drop=True)
    
    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test1_activity_merge['acc_id'].nunique(), test1_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
    train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
    test1_payment_merge = test1_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

    train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량으로 변경
    test1_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test1_payment_merge['acc_id'].nunique(), test1_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test1_combat['class'].unique():test1_combat[class_names[elem]] = test1_combat['class'] == elem
    
    combat_agg = {'day':'nunique','char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
    test1_combat_merge = test1_combat.groupby('acc_id', as_index = False).agg(combat_agg).reset_index(drop=True)
  
    train_combat_merge.rename(columns=combat_rename, inplace=True)
    test1_combat_merge.rename(columns=combat_rename, inplace=True)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test1_combat_merge['acc_id'].nunique(), test1_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'day':'nunique','char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    test1_pledge_merge = test1_pledge.groupby('acc_id', as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test1_pledge_merge['acc_id'].nunique(), test1_pledge_merge.shape)
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left')
        
    test1_merge = test1_activity_merge.merge(
    test1_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test1_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test1_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left')
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test1_merge['acc_id'].nunique(), test1_merge.shape)
    
    
    # fill NA
#     fill_NA(train_merge)
    
    # encode features
#     encode_features(train_merge)
    
### STEP1.1 : prepare train and vaild data

def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape)
    print('train ys shape: ', train_ys.shape)
    
    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape)
    print('valid ys shape: ', valid_ys.shape)
    
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test1_merge
    test_acc_id = test1_merge['acc_id']
    test_x.drop(columns=['acc_id'], inplace=True)
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params
    xgb_params = {
        'eta': 0.007,
        'max_depth': 6, 
        'subsample': 0.6,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }
    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=10000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=10)

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=10000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=10)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    filename = 'test1_predict.csv'
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [1 if i < 1 else i for i in test_predict['amount_spent']]
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']]
    
    test_predict.to_csv(submission_path + filename,index=False)    
 

### etc

In [0]:

from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb

from datetime import datetime
import gc


from math import sqrt


import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    
    testname = 'test2'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/nes_inference/'
        
    def __str__(self):
        return self.value


In [0]:
a = test_trade['source_acc_id'].unique()
b = test_acc_id
len(set(a) & set(b))     # & is intersection - elements common to both

12362

In [0]:
test1_predict = pd.read_csv(submission_path + 'test1_predict.csv')
test1_predict['amount_spent'] = test1_predict['amount_spent']*16
test1_predict.to_csv(submission_path + 'test1_predict_t.csv',index=False) 

test2_predict = pd.read_csv(submission_path + 'test2_predict.csv')
test2_predict['amount_spent'] = test2_predict['amount_spent']*3
test2_predict.to_csv(submission_path + 'test2_predict_t.csv',index=False) 

In [0]:
test1_predict['survival_time'] = [63 if i > 63 else i for i in test_predict['survival_time']] # 63 이상 값 63 처리
test1_predict.to_csv(submission_path + testname + '_predict.csv',index=False)    

NameError: ignored

In [0]:
train_merge = pd.read_csv(submission_path  + 'merge/' + 'train_merge.csv')
test1_merge = pd.read_csv(submission_path  + 'merge/' + 'test1_merge.csv')
test1_merge = pd.read_csv(submission_path  + 'merge/' + 'test1_merge.csv')

In [0]:
train_merge['']

Index(['acc_id', 'day', 'char_id', 'server', 'npc_kill', 'solo_exp',
       'party_exp', 'quest_exp', 'rich_monster', 'death', 'exp_recovery',
       'private_shop', 'game_money_change', 'game_money_change_minus',
       'enchant_count', 'level', 'pledge_cnt', 'liege', 'pledge_id',
       'pledge_combat_cnt', 'combat_play_time', 'non_combat_play_time',
       'max_spent', 'mean_spent', 'trade_day_x', 'trade_server_x',
       'item_amount_x', 't23-02_x', 't03-06_x', 't07-10_x', 't11-14_x',
       't15-18_x', 't19-22_x', 'item_price_x', 'trade_shop_x',
       'personal_shop_x', 'accessory_x', 'adena_x', 'armor_x',
       'enchant_scroll_x', 'etc_x', 'spell_x', 'weapon_x', 'source',
       'trade_day_y', 'trade_server_y', 'item_amount_y', 't23-02_y',
       't03-06_y', 't07-10_y', 't11-14_y', 't15-18_y', 't19-22_y',
       'item_price_y', 'trade_shop_y', 'personal_shop_y', 'accessory_y',
       'adena_y', 'armor_y', 'enchant_scroll_y', 'etc_y', 'spell_y',
       'weapon_y', 'target', 'att

In [0]:
train_payment = pd.read_csv(train_path + 'train_payment.csv')
test1_payment = pd.read_csv(test_path + 'test1' + '_payment.csv')
test2_payment = pd.read_csv(test_path + 'test2' + '_payment.csv')

In [0]:
print(train_payment['amount_spent'].min(),train_payment['amount_spent'].mean(),train_payment['amount_spent'].max())
print(test1_payment['amount_spent'].min(),test1_payment['amount_spent'].mean(),test1_payment['amount_spent'].max())
print(test2_payment['amount_spent'].min(),test2_payment['amount_spent'].mean(),test2_payment['amount_spent'].max())

0.01173470155204053 0.6681977124565469 11.733528081885328
0.01173470155204053 0.7679355719841668 10.561231396836476
0.01173470155204053 0.8403611243368606 11.73470155204053
